Our First (Toy) Fuzzer

In [1]:
import random
def toy_fuzzer(max_length=100, char_start=32, char_range=32):
    """A string of up to `max_length` characters
       in the range [`char_start`, `char_start` + `char_range`]"""
    string_length = random.randrange(0, max_length + 1)
    out = ""
    for i in range(0, string_length):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

In [2]:
N = 3
for i in range(N):
    print("Run "+str(i+1)+":")
    print(toy_fuzzer())
    print()

Run 1:
<9.2!4

Run 2:
).0/%4+0"4)1'07$>/?/6./>2#''5'<,*#825?40-5.56(.)2>22)).$-7*2>

Run 3:
?>!? ?**3-5> +9/9*%7'9,5%2='-,-!/"5<76/39(6?(375,&<>.:"2#$->"



Let's Talk About "Code Coverage"

CGI encoding is a technique used in URLs (i.e., Web addresses) to encode characters that would be invalid in a URL, such as blanks and certain punctuation. 

Example: go to the CHR website, https://www.chrobinson.com/en-US/, and search for "latest news". The URL you get is "https://www.chrobinson.com/en-US/Search/?keyword=%20latest%20news". What's this %20 crap? It's CGI encoding of space.

I found a CGI decoding function online.

In [1]:
def cgi_decode(s):
    """Decode the CGI-encoded string `s`:
       * replace "+" by " "
       * replace "%xx" by the character with hex number xx.
       Return the decoded string.  Raise `ValueError` for invalid inputs."""

    # Mapping of hex digits to their integer values
    hex_values = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15,
        'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
    }

    t = ""
    i = 0
    while i < len(s):
        c = s[i]
        if c == '+':
            t += ' '
        elif c == '%':
            digit_high, digit_low = s[i + 1], s[i + 2]
            i += 2
            if digit_high in hex_values and digit_low in hex_values:
                v = hex_values[digit_high] * 16 + hex_values[digit_low]
                t += chr(v)
            else:
                raise ValueError("Invalid encoding")
        else:
            t += c
        i += 1
    return t

In [4]:
cgi_decode("Hello+world")

'Hello world'

In [5]:
cgi_decode("keyword=%20latest%20news")

'keyword= latest news'

It looks to be working. Cool. I guess my work here is done.

Oh wait. If the CHR website crashed and while doing so dumped out all of our customers' private information CHR would be in a whole lot of trouble...

Let's start testing stuff:

In [6]:
assert cgi_decode('+') == ' '
assert cgi_decode('%20') == ' '
assert cgi_decode('abc') == 'abc'

try:
    cgi_decode('%?a')
    assert False
except ValueError:
    pass

Tests pass. But what if I missed something? Is there a way to systematically test this CGI decoder function without tediously compiling together a huge number of test cases by hand?

Let's try our toy fuzzer.

In [7]:
for i in range(N):
    print("Run "+str(i+1)+":")
    try:
        print(cgi_decode(toy_fuzzer()))
        print()
    except ValueError:
        print("Value Error")

Run 1:
Value Error
Run 2:
Value Error
Run 3:
Value Error


Coolio... Did we cover all the cases we need to cover? Don't know.

Let's think about code coverage. That is, what percentage of the source code is actually executed.

Let's trace an example execution using sys.settrace(). At each line being executed, we are going to obtain the function name, line number and local variables / assignments.

In [2]:
coverage = []
import sys
def traceit(frame, event, arg):
    if event == "line":
        global coverage
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        coverage.append(lineno)
    return traceit

Event here is a string with values including "line" (a new line has been reached) or "call" (a function is being called).

In [3]:
def cgi_decode_traced(s):
    global coverage
    coverage = []
    sys.settrace(traceit)  # Turn on
    cgi_decode(s)
    sys.settrace(None)    # Turn off

Ok, let's trace:

In [4]:
cgi_decode_traced("a+b")
print(coverage)

[9, 10, 11, 12, 15, 16, 17, 18, 19, 21, 30, 31, 17, 18, 19, 20, 31, 17, 18, 19, 21, 30, 31, 17, 32]


In [11]:
import inspect
cgi_decode_code = inspect.getsource(cgi_decode)
cgi_decode_lines = [""] + cgi_decode_code.splitlines()

In [12]:
for i in range(len(cgi_decode_lines)):
    line = cgi_decode_lines[i]
    if (i in coverage):
        print(line)
    else:
        print("#"+line)

#
#def cgi_decode(s):
#    """Decode the CGI-encoded string `s`:
#       * replace "+" by " "
#       * replace "%xx" by the character with hex number xx.
#       Return the decoded string.  Raise `ValueError` for invalid inputs."""
#
#    # Mapping of hex digits to their integer values
#    hex_values = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15,
        'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
#    }
#
    t = ""
    i = 0
    while i < len(s):
        c = s[i]
        if c == '+':
            t += ' '
        elif c == '%':
#            digit_high, digit_low = s[i + 1], s[i + 2]
#            i += 2
#            if digit_high in hex_values and digit_low in hex_values:
#                v = hex_values[digit_high] * 16 + hex_values[digit_low]
#                t += chr(v)
#            else:
#                raise ValueError("Invalid encoding")
#        else:
  

We can see that we've covered some portion of the code, but certainly not all lines. We'll come back to improving our code coverage later. For now, let's see a demo.

In [13]:
trials = 100
for i in range(trials):
    try:
        s = toy_fuzzer()
        cgi_decode(s)
    except ValueError:
        pass

In [14]:
s

'0-0&!!-;<+(5/;<+7 $6,#5<,'

In [15]:
cgi_decode_traced(s)

In [38]:
import inspect
cgi_decode_code = inspect.getsource(cgi_decode)
cgi_decode_lines = [""] + cgi_decode_code.splitlines()

In [39]:
for i in range(len(cgi_decode_lines)):
    line = cgi_decode_lines[i]
    if (i in coverage):
        print(line)
    else:
        print("#"+line)

TypeError: argument of type 'module' is not iterable

Can you see what the problem is?

At this point, you can try to create some more test cases by hand. But we're data scientists. Let's see if we can leverage our skills to create an intelligent solution in harder situations.

Let us now assume we have a program that takes a URL as input. 

In [18]:
from urllib.parse import urlparse
def http_program(url):
    supported_schemes = ["http", "https"]
    result = urlparse(url)
    if result.scheme not in supported_schemes:
        raise ValueError("Scheme must be one of " + repr(supported_schemes))
    if result.netloc == '':
        raise ValueError("Host must be non-empty")

    # Do something with the URL
    return True

If we use our toy fuzzer, the odds of getting a valid URL are minute. So just randomly generating inputs is unlikely to allow us to test the program.

Instead, we are going to start with a valid URL, and then mutate it (think Genetic Algorithms).

Here are a few types of mutations we'll use.

In [19]:
import random
def delete_random_character(s):
    """Returns s with a random character deleted"""
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    # print("Deleting", repr(s[pos]), "at", pos)
    return s[:pos] + s[pos + 1:]

def insert_random_character(s):
    """Returns s with a random character inserted"""
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 127))
    # print("Inserting", repr(random_character), "at", pos)
    return s[:pos] + random_character + s[pos:]

def flip_random_character(s):
    """Returns s with a random bit flipped in a random position"""
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    c = s[pos]
    bit = 1 << random.randint(0, 6)
    new_c = chr(ord(c) ^ bit)
    # print("Flipping", bit, "in", repr(c) + ", giving", repr(new_c))
    return s[:pos] + new_c + s[pos + 1:]

This is our mutator, that chooses a mutation at random.

In [20]:
def mutate(s):
    """Return s with a random mutation applied"""
    mutators = [
        delete_random_character,
        insert_random_character,
        flip_random_character
    ]
    mutator = random.choice(mutators)
    # print(mutator)
    return mutator(s)

In [21]:
for i in range(10):
    print(mutate("A quick brown fox"))

A quick brown fx
A quck brown fox
A quiCk brown fox
A quic+ brown fox
A$quick brown fox
A quik brown fox
A quick brown fox
A quick brown &ox
A quick brosn fox
A quiXck brown fox


Now, we'll be applying this to URLs. Let's define a function to check if a URL is valid.

In [22]:
def is_valid_url(url):
    try:
        result = http_program(url)
        return True
    except ValueError:
        return False
assert is_valid_url("http://www.google.com/search?q=fuzzing")
assert not is_valid_url("xyzzy")

But doe this function actually cover all possible cases?

In [23]:
coverage = set([])
import sys
def traceit(frame, event, arg):
    if event == "line":
        global coverage
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        coverage.add(lineno)
    coverage = set(coverage) # I'm interested now in the set of lines, not sequence of lines
    return traceit

def http_verify_traced(s):
    global coverage
    coverage = set([])
    sys.settrace(traceit)  # Turn on
    is_valid_url(s)
    sys.settrace(None)    # Turn off

In [24]:
class MutationFuzzer():
    def __init__(self, seed, min_mutations=2, max_mutations=10):
        self.min_mutations = min_mutations
        self.max_mutations = max_mutations
        self.population = []
        self.coverages_seen = set([])
        for initial_candidate in seed:
            self.get_candidate_coverage(initial_candidate)
    
    def mutate(self, inp):
        return mutate(inp)
    
    def create_candidate(self):
        candidate = random.choice(self.population)
        mutations = random.randint(self.min_mutations, self.max_mutations)
        for i in range(mutations):
            candidate = self.mutate(candidate)
        return candidate
    
    def is_valid(self, inp):
        return is_valid_url(inp)
    
    def get_candidate_coverage(self, candidate):
        if (self.is_valid(candidate)):
                coverage = self.score_coverage(candidate)
                if (coverage not in self.coverages_seen):
                    self.population.append(candidate)
                    self.coverages_seen.add(frozenset(coverage))
    
    def fuzz(self, numTrials = 1000):
        for i in range(numTrials):
            candidate = self.create_candidate()
            self.get_candidate_coverage(candidate)
    
    def score_coverage(self, inp):
        http_verify_traced(inp)
        return coverage   

In [25]:
seed_input = "http://www.google.com/search?q=fuzzing"
mutation_fuzzer = MutationFuzzer(seed=[seed_input])

In [26]:
mutation_fuzzer.population

['http://www.google.com/search?q=fuzzing']

In [27]:
mutation_fuzzer.coverages_seen

{frozenset({1,
            2,
            3,
            4,
            5,
            7,
            11,
            99,
            115,
            116,
            119,
            121,
            122,
            367,
            368,
            369,
            370,
            373,
            374,
            375,
            418,
            419,
            420,
            421,
            422,
            423})}

In [28]:
mutation_fuzzer.fuzz(1000000)

In [29]:
mutation_fuzzer.population

['http://www.google.com/search?q=fuzzing',
 'http://www.google.com/searh;q=fuzzinb2g',
 'http://www.Xgole.com/eaHrh;qfu/zzinb2g']

In [30]:
mutation_fuzzer.coverages_seen

{frozenset({1,
            2,
            3,
            4,
            5,
            7,
            11,
            99,
            115,
            116,
            119,
            121,
            122,
            367,
            368,
            369,
            370,
            373,
            374,
            375,
            418,
            419,
            420,
            421,
            422,
            423}),
 frozenset({1,
            2,
            3,
            4,
            5,
            7,
            11,
            99,
            115,
            116,
            119,
            121,
            122,
            367,
            368,
            369,
            370,
            371,
            374,
            375,
            378,
            379,
            380,
            381,
            418,
            419,
            420,
            421,
            422,
            423}),
 frozenset({1,
            2,
            3,
            4,
            

Interesting side note: fuzzing is being used to debug and test out neural networks. Coverage is measured by determining which neurons get activated.

Now we introduce a new concept, evolutionary fuzzing.

In [31]:
initial_pop = ['http://www.google.com/search?q=fuzzing',
 'http://www.google.com/searh;q=fuzzinb2g',
 'http://www.Xgole.com/eaHrh;qfu/zzinb2g']



In [36]:
import coverage

cov = coverage.Coverage()
cov.start()

cgi_decode("a+b")

cov.stop()
cov.save()

cov.html_report()

Coverage.py warning: No data was collected. (no-data-collected)


CoverageException: No data to report.

In [32]:
def trace_lines(frame, event, arg):
    lines = []
    if event == "line":
        line = frame.f_lineno
        lines.append(lines)
    return lines

import sys
x = sys.settrace(trace_lines)  # Turn on
cgi_decode(s)
sys.settrace(None)  

TypeError: 'list' object is not callable

In [3]:
def cgi_decode(s):
    """Decode the CGI-encoded string `s`:
       * replace "+" by " "
       * replace "%xx" by the character with hex number xx.
       Return the decoded string.  Raise `ValueError` for invalid inputs.
       """

    # Mapping of hex digits to their integer values
    hex_values = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15,
        'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
    }

    t = ""
    i = 0
    while i < len(s):
        c = s[i]
        if c == '+':
            t += ' '
        elif c == '%':
            digit_high, digit_low = s[i + 1], s[i + 2]
            i += 2
            if digit_high in hex_values and digit_low in hex_values:
                v = hex_values[digit_high] * 16 + hex_values[digit_low]
                t += chr(v)
            else:
                raise ValueError("Invalid encoding")
        else:
            t += c
        i += 1
    return t

In [2]:
def traceit(frame, event, arg):
    if event == "line":
        global coverage
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        coverage.append(lineno)
    return traceit

In [3]:
import sys
def cgi_decode_traced(s):
    global coverage
    coverage = []
    sys.settrace(traceit)  # Turn on
    cgi_decode(s)
    sys.settrace(None)    # Turn off
    return coverage

In [4]:
cgi_decode_traced("a+b")

[10,
 11,
 12,
 13,
 16,
 17,
 18,
 19,
 20,
 22,
 31,
 32,
 18,
 19,
 20,
 21,
 32,
 18,
 19,
 20,
 22,
 31,
 32,
 18,
 33]

In [1]:
def cgi_decode(s):
    """Decode the CGI-encoded string `s`:
       * replace "+" by " "
       * replace "%xx" by the character with hex number xx.
       Return the decoded string.  Raise `ValueError` for invalid inputs.
       """

    # Mapping of hex digits to their integer values
    hex_values = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4,
        '5': 5, '6': 6, '7': 7, '8': 8, '9': 9,
        'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14, 'f': 15,
        'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
    }

    t = ""
    i = 0
    while i < len(s):
        c = s[i]
        if c == '+':
            t += ' '
        elif c == '%':
            digit_high, digit_low = s[i + 1], s[i + 2]
            i += 2
            if digit_high in hex_values and digit_low in hex_values:
                v = hex_values[digit_high] * 16 + hex_values[digit_low]
                t += chr(v)
            else:
                raise ValueError("Invalid encoding")
        else:
            t += c
        i += 1
    return t

In [2]:
def line_tracer(frame, event, arg):
    if event == "line":
        global coverage
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        coverage.append(lineno)
    return line_tracer

In [3]:
import sys
def record_coverage(function, s):
    global coverage
    coverage = []
    sys.settrace(line_tracer)
    function(s)
    sys.settrace(None)
    return coverage

In [5]:
record_coverage(print, "a+b")

a+b


[383,
 391,
 395,
 398,
 308,
 400,
 200,
 1100,
 1101,
 509,
 1103,
 1057,
 1058,
 1060,
 1104,
 201,
 203,
 96,
 97,
 105,
 385,
 390,
 395,
 401,
 408,
 325,
 327,
 330,
 332,
 200,
 1100,
 1101,
 509,
 1103,
 1057,
 1058,
 1060,
 1104,
 201,
 203,
 96,
 97,
 105,
 385,
 390,
 395,
 383,
 391,
 395,
 398,
 308,
 400,
 200,
 1100,
 1101,
 509,
 1103,
 1057,
 1058,
 1060,
 1104,
 201,
 203,
 96,
 97,
 105,
 385,
 390,
 395,
 401,
 408,
 325,
 326]

One more option is edge coverage. We think of code blocks A, B, C, D, E as nodes in a graph. Then an execution path might be A->B->C->D->E. Another one might be A->B->A. Then the edge B->A is a new edge, and we have new edge coverage in this path. Maximizing edge coverage is another metric that can guide our fuzzers.

Talk about AFL